In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [2]:
x = tf.placeholder(dtype=tf.float32, shape=[1], name="x")
val = x + tf.constant([1.0], dtype=tf.float32)
output = tf.identity(val, name="output")

In [3]:
print(x, output)

Tensor("x:0", shape=(1,), dtype=float32) Tensor("output:0", shape=(1,), dtype=float32)


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    graph_def = tf.get_default_graph().as_graph_def()
    output = graph_util.convert_variables_to_constants(sess, graph_def, ["output"])
    with tf.gfile.GFile("./add_by_one.pb", "wb") as f:
        f.write(output.SerializeToString())

INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.


Build toco:

```
bazel build TENSORFLOW_REPO_DIR/contrib/lite/toco:toco
```

Generate tflite file by toco:
```
TENSORFLOW_REPO_DIR/bazel-bin/tensorflow/contrib/lite/toco/toco \
  --input_file=add_by_one.pb \
  --input_format=TENSORFLOW_GRAPHDEF  --output_format=TFLITE \
  --output_file=add_by_one.tflite --inference_type=FLOAT \
  --input_type=FLOAT --input_arrays=x \
  --output_arrays=output --input_shapes=1
```

Relpace `TENSORFLOW_REPO_DIR` with the location of tensorflow repository.